In [1]:
import tensorflow

In [2]:
import keras

Using TensorFlow backend.


In [3]:
import pandas as pd
import numpy as np

In [4]:
df = pd.read_csv('UNSW_NB15_training-set.csv')

In [5]:
df = df.sample(n=80000, replace=True)

In [6]:
df.shape

(80000, 45)

In [7]:
df.attack_cat.value_counts()

Normal            25517
Generic           18357
Exploits          15139
Fuzzers            8261
DoS                5528
Reconnaissance     4835
Analysis            928
Backdoor            853
Shellcode           516
Worms                66
Name: attack_cat, dtype: int64

In [8]:
# find categorical variables
categorical = [var for var in df.columns if df[var].dtype=='O']
print('There are {} categorical variables\n'.format(len(categorical)))
print('The categorical variables are :', categorical)
# check for cardinality in categorical variables
for var in categorical:    
    print(var, ' contains ', len(df[var].unique()), ' labels')

There are 4 categorical variables

The categorical variables are : ['proto', 'service', 'state', 'attack_cat']
proto  contains  132  labels
service  contains  13  labels
state  contains  6  labels
attack_cat  contains  10  labels


In [9]:
#new
df['proto'] = df['proto'].astype('category')
df['service'] = df['service'].astype('category')
df['state'] = df['state'].astype('category')
df['attack_cat'] = df['attack_cat'].astype('category')
cat_columns = df.select_dtypes(['category']).columns
df[cat_columns] = df[cat_columns].apply(lambda x: x.cat.codes)

In [10]:
# Categorical boolean mask
categorical_feature_mask = df.dtypes==object
# filter categorical columns using mask and turn it into a list
categorical_cols = df.columns[categorical_feature_mask].tolist()

In [11]:
X = df.iloc[:, df.columns!='attack_cat']
Y = df.iloc[:, 43]

In [12]:
names_inputed = ['id',
 'dur',
 'proto',
 'service',
 'state',
 'spkts',
 'dpkts',
 'sbytes',
 'dbytes',
 'rate',
 'sttl',
 'dttl',
 'sload',
 'dload',
 'sloss',
 'dloss',
 'sinpkt',
 'dinpkt',
 'sjit',
 'djit',
 'swin',
 'stcpb',
 'dtcpb',
 'dwin',
 'tcprtt',
 'synack',
 'ackdat',
 'smean',
 'dmean',
 'trans_depth',
 'response_body_len',
 'ct_srv_src',
 'ct_state_ttl',
 'ct_dst_ltm',
 'ct_src_dport_ltm',
 'ct_dst_sport_ltm',
 'ct_dst_src_ltm',
 'is_ftp_login',
 'ct_ftp_cmd',
 'ct_flw_http_mthd',
 'ct_src_ltm',
 'ct_srv_dst',
 'is_sm_ips_ports',
 'label']

In [13]:
#StandardScaler
from sklearn.preprocessing import StandardScaler
sScaler = StandardScaler()
rescaleX = sScaler.fit_transform(X)
df_rescaled = pd.DataFrame(data=rescaleX, columns=names_inputed)

In [14]:
#normalization
from sklearn.preprocessing import Normalizer
norm = Normalizer()
xNormalize = norm.fit_transform(X)
df_Normalized = pd.DataFrame(data=xNormalize, columns=names_inputed)

In [15]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(rescaleX, Y, test_size=0.3)

In [16]:
from keras.utils import to_categorical
y_train = to_categorical(y_train)

In [17]:
from keras.models import Sequential

In [18]:
from keras.layers import Dense

In [80]:
#Initialising the Artificial Neural Network
model = Sequential()
#Creating the Input-layer and the first hidden layer
model.add(Dense(400, input_dim=44, activation='sigmoid'))
model.add(Dense(800, input_dim=44, activation='sigmoid'))
model.add(Dense(800, input_dim=44, activation='sigmoid'))
model.add(Dense(400, input_dim=44, activation='sigmoid'))
#Creating the output  layer
model.add(Dense(10, activation='softmax'))
#Compiling the ANN classifier
model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])
model.fit(X_train, y_train, batch_size=50, epochs=100)

Epoch 1/100
56000/56000 [==============================] - 15s 273us/step - loss: 0.7240 - accuracy: 0.7374
Epoch 2/100
56000/56000 [==============================] - 16s 286us/step - loss: 0.5728 - accuracy: 0.7878
Epoch 3/100
56000/56000 [==============================] - 16s 282us/step - loss: 0.5207 - accuracy: 0.8069
Epoch 4/100
56000/56000 [==============================] - 16s 283us/step - loss: 0.4829 - accuracy: 0.8251
Epoch 5/100
56000/56000 [==============================] - 15s 262us/step - loss: 0.4619 - accuracy: 0.8326
Epoch 6/100
56000/56000 [==============================] - 13s 240us/step - loss: 0.4487 - accuracy: 0.8361
Epoch 7/100
56000/56000 [==============================] - 14s 242us/step - loss: 0.4430 - accuracy: 0.8389s -
Epoch 8/100
56000/56000 [==============================] - 13s 239us/step - loss: 0.4370 - accuracy: 0.8394
Epoch 9/100
56000/56000 [==============================] - 14s 245us/step - loss: 0.4340 - accuracy: 0.8412
Epoch 10/100
56000/56000 

In [81]:
y_test = to_categorical(y_test)

In [82]:
predictions = model.predict(X_test)
model.evaluate(X_test,y_test)

24000/24000 [==============================] - 3s 119us/step


[0.4308823263148467, 0.8480833172798157]

In [83]:
from sklearn import metrics
con = metrics.confusion_matrix(y_test.argmax(axis=1), predictions.argmax(axis=1))

In [91]:
precision = np.diag(con) / (np.diag(con) +(con.sum(axis=0) - np.diag(con)))
precision.mean()

D:\Anaconda\lib\site-packages\ipykernel_launcher.py:1: RuntimeWarning: invalid value encountered in true_divide
  """Entry point for launching an IPython kernel.


array([0.82191781, 0.        , 0.36453202, 0.7055989 , 0.84796238,
       0.98821146, 1.        , 0.85185185, 0.55555556,        nan])

In [92]:
recall = np.diag(con)/ ((con.sum(axis=1) - np.diag(con)) + np.diag(con))
recall.mean()

0.532849723225898

In [86]:
f1_score = 2 * ((precision * recall) / (precision + recall))
f1_score

D:\Anaconda\lib\site-packages\ipykernel_launcher.py:1: RuntimeWarning: invalid value encountered in true_divide
  """Entry point for launching an IPython kernel.


array([0.3030303 ,        nan, 0.44007319, 0.6905457 , 0.86283892,
       0.98594138, 0.99967322, 0.77094972, 0.42654028,        nan])

In [69]:
#InfoGainAtributeEval
features = ['sttl','ct_state_ttl','ct_flw_http_mthd','sbytes','id','smean','sload','dur','sinpkt','rate','proto','ct_dst_src_ltm','service','dbytes','sjit','ct_srv_dst','dload','dinpkt','dmean','ct_srv_src','synack','tcprtt','ct_dst_sport_ltm','djit','ct_src_dport_ltm','dtcpb','stcpb','spkts','dloss','ct_dst_ltm','ackdat','label','dpkts','ct_src_ltm','sloss']
X2 = df[['sttl','ct_state_ttl','ct_flw_http_mthd','sbytes','id','smean','sload','dur','sinpkt','rate','proto','ct_dst_src_ltm','service','dbytes','sjit','ct_srv_dst','dload','dinpkt','dmean','ct_srv_src','synack','tcprtt','ct_dst_sport_ltm','djit','ct_src_dport_ltm','dtcpb','stcpb','spkts','dloss','ct_dst_ltm','ackdat','label','dpkts','ct_src_ltm','sloss']]
Y2 = df[['attack_cat']]
X3 = X2.iloc[:,0:35]  #independent columns
Y3 = Y2.iloc[:,-1]    #target column 

In [18]:
from sklearn.preprocessing import StandardScaler
sScaler = StandardScaler()
rescaleX3 = sScaler.fit_transform(X3)
df_rescaled = pd.DataFrame(data=rescaleX3, columns=features)

In [19]:
X_train, X_test, y_train, y_test = train_test_split(rescaleX3, Y3, test_size=0.3)

In [20]:
y_train = to_categorical(y_train)

In [21]:
#Initialising the Artificial Neural Network
model = Sequential()
#Creating the Input-layer and the first hidden layer
model.add(Dense(300, input_dim=35, activation='sigmoid'))
model.add(Dense(600, input_dim=35, activation='sigmoid'))
model.add(Dense(600, input_dim=35, activation='sigmoid'))
model.add(Dense(300, input_dim=35, activation='sigmoid'))
#Creating the output  layer
model.add(Dense(10, activation='softmax'))
#Compiling the ANN classifier
model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])
model.fit(X_train, y_train, batch_size=50, epochs=100)

Epoch 1/100
56000/56000 [==============================] - 16s 282us/step - loss: 0.7354 - accuracy: 0.7328s - ETA: 1s - loss: 0.7474 - accuracy
Epoch 2/100
56000/56000 [==============================] - ETA: 0s - loss: 0.5785 - accuracy: 0.78 - 15s 270us/step - loss: 0.5784 - accuracy: 0.7865
Epoch 3/100
56000/56000 [==============================] - 16s 289us/step - loss: 0.5363 - accuracy: 0.8020
Epoch 4/100
56000/56000 [==============================] - 17s 304us/step - loss: 0.5043 - accuracy: 0.8175
Epoch 5/100
56000/56000 [==============================] - 16s 294us/step - loss: 0.4820 - accuracy: 0.8265
Epoch 6/100
56000/56000 [==============================] - 17s 295us/step - loss: 0.4666 - accuracy: 0.8318s - loss: 0.4670 - 
Epoch 7/100
56000/56000 [==============================] - 17s 296us/step - loss: 0.4546 - accuracy: 0.8353
Epoch 8/100
56000/56000 [==============================] - 17s 301us/step - loss: 0.4461 - accuracy: 0.8371
Epoch 9/100
56000/56000 [=============

In [22]:
y_test = to_categorical(y_test)

In [23]:
predictions = model.predict(X_test)
model.evaluate(X_test,y_test)

24000/24000 [==============================] - 2s 100us/step


[0.4123147657414277, 0.8571666479110718]

In [30]:
from sklearn import metrics
con = metrics.confusion_matrix(y_test.argmax(axis=1), predictions.argmax(axis=1))

In [33]:
precision = np.diag(con) / (np.diag(con) +(con.sum(axis=0) - np.diag(con)))
precision

D:\Anaconda\lib\site-packages\ipykernel_launcher.py:1: RuntimeWarning: invalid value encountered in true_divide
  """Entry point for launching an IPython kernel.


array([0.66666667, 0.3       , 0.35147745, 0.63175992, 0.89406237,
       0.99063401, 1.        , 0.86736475, 0.3815261 ,        nan])

In [32]:
recall = np.diag(con)/ ((con.sum(axis=1) - np.diag(con)) + np.diag(con))
recall

array([0.29370629, 0.01485149, 0.13839559, 0.87037037, 0.8487429 ,
       0.98495702, 0.9998673 , 0.66577361, 0.69343066, 0.        ])

In [34]:
f1_score = 2 * ((precision * recall) / (precision + recall))
f1_score

array([0.40776699, 0.02830189, 0.19859402, 0.73211375, 0.8708134 ,
       0.98778736, 0.99993365, 0.75331565, 0.49222798,        nan])

In [35]:
#CorrelationAttributeEval
features = ['id','ct_dst_sport_ltm','ct_dst_src_ltm','ct_src_dport_ltm','sttl','ct_srv_dst','ct_srv_src','ct_dst_ltm','ct_src_ltm','ct_state_ttl','state','swin','dwin','proto','service','rate','dttl','stcpb','dtcpb','dmean','dload','tcprtt','ackdat','synack']
X2 = df[['id','ct_dst_sport_ltm','ct_dst_src_ltm','ct_src_dport_ltm','sttl','ct_srv_dst','ct_srv_src','ct_dst_ltm','ct_src_ltm','ct_state_ttl','state','swin','dwin','proto','service','rate','dttl','stcpb','dtcpb','dmean','dload','tcprtt','ackdat','synack']]
Y2 = df[['attack_cat']]
X3 = X2.iloc[:,0:24]  #independent columns
Y3 = Y2.iloc[:,-1]    #target column 

In [36]:
sScaler = StandardScaler()
rescaleX3 = sScaler.fit_transform(X3)
df_rescaled = pd.DataFrame(data=rescaleX3, columns=features)

In [37]:
X_train, X_test, y_train, y_test = train_test_split(rescaleX3, Y3, test_size=0.3)

In [38]:
y_train = to_categorical(y_train)

In [39]:
#Initialising the Artificial Neural Network
model = Sequential()
#Creating the Input-layer and the first hidden layer
model.add(Dense(200, input_dim=24, activation='sigmoid'))
model.add(Dense(400, input_dim=24, activation='sigmoid'))
model.add(Dense(400, input_dim=24, activation='sigmoid'))
model.add(Dense(200, input_dim=24, activation='sigmoid'))
#Creating the output  layer
model.add(Dense(10, activation='softmax'))
#Compiling the ANN classifier
model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])
model.fit(X_train, y_train, batch_size=50, epochs=100)

Epoch 1/100
56000/56000 [==============================] - 9s 159us/step - loss: 0.8981 - accuracy: 0.6719
Epoch 2/100
56000/56000 [==============================] - 8s 145us/step - loss: 0.7120 - accuracy: 0.7351
Epoch 3/100
56000/56000 [==============================] - 8s 143us/step - loss: 0.6689 - accuracy: 0.7490
Epoch 4/100
56000/56000 [==============================] - 8s 144us/step - loss: 0.6455 - accuracy: 0.7590
Epoch 5/100
56000/56000 [==============================] - 8s 145us/step - loss: 0.6303 - accuracy: 0.7660
Epoch 6/100
56000/56000 [==============================] - 8s 146us/step - loss: 0.6172 - accuracy: 0.7703
Epoch 7/100
56000/56000 [==============================] - 8s 146us/step - loss: 0.6091 - accuracy: 0.7739
Epoch 8/100
56000/56000 [==============================] - 8s 148us/step - loss: 0.6010 - accuracy: 0.7774
Epoch 9/100
56000/56000 [==============================] - 8s 148us/step - loss: 0.5951 - accuracy: 0.7802
Epoch 10/100
56000/56000 [===========

In [40]:
y_test = to_categorical(y_test)
predictions = model.predict(X_test)
model.evaluate(X_test,y_test)

24000/24000 [==============================] - 2s 80us/step


[0.5547136311531067, 0.8036249876022339]

In [41]:
from sklearn import metrics
con = metrics.confusion_matrix(y_test.argmax(axis=1), predictions.argmax(axis=1))

In [42]:
precision = np.diag(con) / (np.diag(con) +(con.sum(axis=0) - np.diag(con)))
precision

D:\Anaconda\lib\site-packages\ipykernel_launcher.py:1: RuntimeWarning: invalid value encountered in true_divide
  """Entry point for launching an IPython kernel.


array([0.59090909, 0.23423423, 0.37115215, 0.61489362, 0.73997413,
       0.99872123, 0.98104265, 0.51195586,        nan,        nan])

In [43]:
recall = np.diag(con)/ ((con.sum(axis=1) - np.diag(con)) + np.diag(con))
recall

array([0.09848485, 0.11016949, 0.24563446, 0.82012661, 0.69983687,
       0.97869674, 0.92972259, 0.57986111, 0.        , 0.        ])

In [44]:
f1_score = 2 * ((precision * recall) / (precision + recall))
f1_score

array([0.16883117, 0.14985591, 0.29562172, 0.70283416, 0.71934605,
       0.98860759, 0.95469343, 0.54379681,        nan,        nan])

In [48]:
#PrincipalComponentAnalysis
features = ['id','dur','dwin','proto','djit','swin','smean','state','service','ct_src_dport_ltm','ct_dst_ltm','ct_dst_sport_ltm','ct_src_ltm','ct_flw_http_mthd','ct_srv_dst','dpkts','dloss','dbytes','sttl','dmean','spkts','sbytes','sloss','sinpkt','label']
X2 = df[['id','dur','dwin','proto','djit','swin','smean','state','service','ct_src_dport_ltm','ct_dst_ltm','ct_dst_sport_ltm','ct_src_ltm','ct_flw_http_mthd','ct_srv_dst','dpkts','dloss','dbytes','sttl','dmean','spkts','sbytes','sloss','sinpkt','label']]
Y2 = df[['attack_cat']]
X3 = X2.iloc[:,0:25]  #independent columns
Y3 = Y2.iloc[:,-1]    #target column 

In [49]:
sScaler = StandardScaler()
rescaleX3 = sScaler.fit_transform(X3)
df_rescaled = pd.DataFrame(data=rescaleX3, columns=features)

In [50]:
X_train, X_test, y_train, y_test = train_test_split(rescaleX3, Y3, test_size=0.3)

In [51]:
y_train = to_categorical(y_train)

In [52]:
#Initialising the Artificial Neural Network
model = Sequential()
#Creating the Input-layer and the first hidden layer
model.add(Dense(200, input_dim=25, activation='sigmoid'))
model.add(Dense(400, input_dim=25, activation='sigmoid'))
model.add(Dense(400, input_dim=25, activation='sigmoid'))
model.add(Dense(200, input_dim=25, activation='sigmoid'))
#Creating the output  layer
model.add(Dense(10, activation='softmax'))
#Compiling the ANN classifier
model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])
model.fit(X_train, y_train, batch_size=50, epochs=100)

Epoch 1/100
56000/56000 [==============================] - 10s 175us/step - loss: 0.7400 - accuracy: 0.7329
Epoch 2/100
56000/56000 [==============================] - 9s 164us/step - loss: 0.5890 - accuracy: 0.7834
Epoch 3/100
56000/56000 [==============================] - 9s 166us/step - loss: 0.5575 - accuracy: 0.7864
Epoch 4/100
56000/56000 [==============================] - 9s 166us/step - loss: 0.5341 - accuracy: 0.7941
Epoch 5/100
56000/56000 [==============================] - 9s 162us/step - loss: 0.5139 - accuracy: 0.8056
Epoch 6/100
56000/56000 [==============================] - 9s 162us/step - loss: 0.4968 - accuracy: 0.8151
Epoch 7/100
56000/56000 [==============================] - 9s 163us/step - loss: 0.4787 - accuracy: 0.8247
Epoch 8/100
56000/56000 [==============================] - 9s 163us/step - loss: 0.4672 - accuracy: 0.8302
Epoch 9/100
56000/56000 [==============================] - 9s 163us/step - loss: 0.4567 - accuracy: 0.8332
Epoch 10/100
56000/56000 [==========

In [53]:
model.evaluate(X_train, y_train)

56000/56000 [==============================] - 2s 37us/step


[0.3890355060015406, 0.852446436882019]

In [54]:
y_test = to_categorical(y_test)

In [55]:
predictions = model.predict(X_test)
model.evaluate(X_test,y_test)

24000/24000 [==============================] - 1s 32us/step


[0.40761351148287456, 0.8500833511352539]

In [56]:
from sklearn import metrics
con = metrics.confusion_matrix(y_test.argmax(axis=1), predictions.argmax(axis=1))

In [57]:
precision = np.diag(con) / (np.diag(con) +(con.sum(axis=0) - np.diag(con)))
precision

D:\Anaconda\lib\site-packages\ipykernel_launcher.py:1: RuntimeWarning: invalid value encountered in true_divide
  """Entry point for launching an IPython kernel.


array([0.57342657, 0.39285714, 0.37982973, 0.67854376, 0.8202765 ,
       0.98191355, 1.        , 0.76688938, 0.53731343,        nan])

In [58]:
recall = np.diag(con)/ ((con.sum(axis=1) - np.diag(con)) + np.diag(con))
recall

array([0.29078014, 0.05288462, 0.3427896 , 0.7550097 , 0.8700611 ,
       0.9842277 , 0.9997366 , 0.70319946, 0.27906977, 0.        ])

In [59]:
f1_score = 2 * ((precision * recall) / (precision + recall))
f1_score

array([0.38588235, 0.09322034, 0.36036036, 0.71473738, 0.84443566,
       0.98306926, 0.99986828, 0.73366477, 0.36734694,        nan])

In [19]:
from sklearn.utils import resample
df_minority0 = df[df.attack_cat==0]
df_minority1 = df[df.attack_cat==1]
df_minority2 = df[df.attack_cat==2]
df_minority3 = df[df.attack_cat==3]
df_minority7 = df[df.attack_cat==7]
df_minority8 = df[df.attack_cat==8]
df_minority9 = df[df.attack_cat==9]

In [20]:
df_minority_upsampled0 = resample(df_minority0, 
                                 replace=True,     # sample with replacement
                                 n_samples=5000)    # to match majority class
df_minority_upsampled1 = resample(df_minority1, 
                                 replace=True,     # sample with replacement
                                 n_samples=5000)    # to match majority class
df_minority_upsampled2 = resample(df_minority2, 
                                 replace=True,     # sample with replacement
                                 n_samples=6000)    # to match majority class
df_minority_upsampled3 = resample(df_minority3, 
                                 replace=True,     # sample with replacement
                                 n_samples=6000)    # to match majority class
df_minority_upsampled7 = resample(df_minority7, 
                                 replace=True,     # sample with replacement
                                 n_samples=3000)    # to match majority class
df_minority_upsampled8 = resample(df_minority8, 
                                 replace=True,     # sample with replacement
                                 n_samples=5000)
df_minority_upsampled9 = resample(df_minority9, 
                                 replace=True,     # sample with replacement
                                 n_samples=6000)    # to match majority class
df = pd.concat([df, df_minority_upsampled0, df_minority_upsampled1, df_minority_upsampled2, df_minority_upsampled3, df_minority_upsampled7, df_minority_upsampled8, df_minority_upsampled9])

In [22]:
#Info Gain Attribute Eval(CA)
features = ['sttl','ct_state_ttl','ct_flw_http_mthd','sbytes','id','smean','sload','dur','sinpkt','rate','proto','ct_dst_src_ltm','service','dbytes','sjit','ct_srv_dst','dload','dinpkt','dmean','ct_srv_src','synack','tcprtt','ct_dst_sport_ltm','djit','ct_src_dport_ltm','dtcpb','stcpb','spkts','dloss','ct_dst_ltm','ackdat','label','dpkts','ct_src_ltm','sloss']
X2 = df[['sttl','ct_state_ttl','ct_flw_http_mthd','sbytes','id','smean','sload','dur','sinpkt','rate','proto','ct_dst_src_ltm','service','dbytes','sjit','ct_srv_dst','dload','dinpkt','dmean','ct_srv_src','synack','tcprtt','ct_dst_sport_ltm','djit','ct_src_dport_ltm','dtcpb','stcpb','spkts','dloss','ct_dst_ltm','ackdat','label','dpkts','ct_src_ltm','sloss']]
Y2 = df[['attack_cat']]
X3 = X2.iloc[:,0:35]  #independent columns
Y3 = Y2.iloc[:,-1]    #target column 

In [23]:
from sklearn.preprocessing import StandardScaler
sScaler = StandardScaler()
rescaleX3 = sScaler.fit_transform(X3)
df_rescaled = pd.DataFrame(data=rescaleX3, columns=features)

In [24]:
X_train, X_test, y_train, y_test = train_test_split(rescaleX3, Y3, test_size=0.3)

In [25]:
y_train = to_categorical(y_train)

In [26]:
#Initialising the Artificial Neural Network
model = Sequential()
#Creating the Input-layer and the first hidden layer
model.add(Dense(300, input_dim=35, activation='sigmoid'))
model.add(Dense(600, input_dim=35, activation='sigmoid'))
model.add(Dense(600, input_dim=35, activation='sigmoid'))
model.add(Dense(300, input_dim=35, activation='sigmoid'))
#Creating the output  layer
model.add(Dense(10, activation='softmax'))
#Compiling the ANN classifier
model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])
model.fit(X_train, y_train, batch_size=50, epochs=100)

Epoch 1/100
81200/81200 [==============================] - 20s 245us/step - loss: 1.0653 - accuracy: 0.5957
Epoch 2/100
81200/81200 [==============================] - 18s 227us/step - loss: 0.8234 - accuracy: 0.6733
Epoch 3/100
81200/81200 [==============================] - 19s 230us/step - loss: 0.7418 - accuracy: 0.7045
Epoch 4/100
81200/81200 [==============================] - 18s 227us/step - loss: 0.7053 - accuracy: 0.7185s -
Epoch 5/100
81200/81200 [==============================] - 19s 233us/step - loss: 0.6846 - accuracy: 0.7267
Epoch 6/100
81200/81200 [==============================] - 18s 225us/step - loss: 0.6709 - accuracy: 0.7306
Epoch 7/100
81200/81200 [==============================] - 18s 225us/step - loss: 0.6606 - accuracy: 0.7358
Epoch 8/100
81200/81200 [==============================] - 18s 219us/step - loss: 0.6508 - accuracy: 0.7400
Epoch 9/100
81200/81200 [==============================] - 18s 219us/step - loss: 0.6425 - accuracy: 0.7431
Epoch 10/100
81200/81200 

In [27]:
y_test = to_categorical(y_test)
predictions = model.predict(X_test)
model.evaluate(X_test,y_test)

34800/34800 [==============================] - 4s 103us/step


[0.6244542399357105, 0.7737069129943848]

In [29]:
from sklearn import metrics
con = metrics.confusion_matrix(y_test.argmax(axis=1), predictions.argmax(axis=1))
precision = np.diag(con) / (np.diag(con) +(con.sum(axis=0) - np.diag(con)))
precision

array([0.71733668, 0.56230366, 0.3566879 , 0.65609158, 0.87564103,
       0.99618598, 0.99986902, 0.77764706, 0.67027281, 0.94886076])

In [30]:
recall = np.diag(con)/ ((con.sum(axis=1) - np.diag(con)) + np.diag(con))
recall

array([0.32114736, 0.31440281, 0.58519594, 0.6381422 , 0.82322218,
       0.98086552, 0.99973808, 0.56495726, 0.86311326, 1.        ])

In [31]:
f1_score = 2 * ((precision * recall) / (precision + recall))
f1_score

array([0.44366744, 0.40330454, 0.44322304, 0.64699242, 0.8486229 ,
       0.98846639, 0.99980355, 0.65445545, 0.75456712, 0.97375942])

In [32]:
#CorrelationAttributeEval
features = ['id','ct_dst_sport_ltm','ct_dst_src_ltm','ct_src_dport_ltm','sttl','ct_srv_dst','ct_srv_src','ct_dst_ltm','ct_src_ltm','ct_state_ttl','state','swin','dwin','proto','service','rate','dttl','stcpb','dtcpb','dmean','dload','tcprtt','ackdat','synack']
X2 = df[['id','ct_dst_sport_ltm','ct_dst_src_ltm','ct_src_dport_ltm','sttl','ct_srv_dst','ct_srv_src','ct_dst_ltm','ct_src_ltm','ct_state_ttl','state','swin','dwin','proto','service','rate','dttl','stcpb','dtcpb','dmean','dload','tcprtt','ackdat','synack']]
Y2 = df[['attack_cat']]
X3 = X2.iloc[:,0:24]  #independent columns
Y3 = Y2.iloc[:,-1]    #target column 

In [33]:
from sklearn.preprocessing import StandardScaler
sScaler = StandardScaler()
rescaleX3 = sScaler.fit_transform(X3)
df_rescaled = pd.DataFrame(data=rescaleX3, columns=features)

In [34]:
X_train, X_test, y_train, y_test = train_test_split(rescaleX3, Y3, test_size=0.3)

In [35]:
y_train = to_categorical(y_train)

In [36]:
#Initialising the Artificial Neural Network
model = Sequential()
#Creating the Input-layer and the first hidden layer
model.add(Dense(200, input_dim=24, activation='sigmoid'))
model.add(Dense(400, input_dim=24, activation='sigmoid'))
model.add(Dense(400, input_dim=24, activation='sigmoid'))
model.add(Dense(200, input_dim=24, activation='sigmoid'))
#Creating the output  layer
model.add(Dense(10, activation='softmax'))
#Compiling the ANN classifier
model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])
model.fit(X_train, y_train, batch_size=50, epochs=100)

Epoch 1/100
81200/81200 [==============================] - 11s 139us/step - loss: 1.2281 - accuracy: 0.5415
Epoch 2/100
81200/81200 [==============================] - 11s 132us/step - loss: 1.0027 - accuracy: 0.6049
Epoch 3/100
81200/81200 [==============================] - 11s 135us/step - loss: 0.9415 - accuracy: 0.6268
Epoch 4/100
81200/81200 [==============================] - 11s 129us/step - loss: 0.9114 - accuracy: 0.6364
Epoch 5/100
81200/81200 [==============================] - 11s 135us/step - loss: 0.8906 - accuracy: 0.6430
Epoch 6/100
81200/81200 [==============================] - 12s 151us/step - loss: 0.8748 - accuracy: 0.6480
Epoch 7/100
81200/81200 [==============================] - 13s 164us/step - loss: 0.8614 - accuracy: 0.6532
Epoch 8/100
81200/81200 [==============================] - 13s 161us/step - loss: 0.8493 - accuracy: 0.6588
Epoch 9/100
81200/81200 [==============================] - 12s 148us/step - loss: 0.8359 - accuracy: 0.6629
Epoch 10/100
81200/81200 [==

In [37]:
y_test = to_categorical(y_test)
predictions = model.predict(X_test)
model.evaluate(X_test,y_test)

34800/34800 [==============================] - 2s 66us/step


[0.7400206503100779, 0.7176724076271057]

In [38]:
con = metrics.confusion_matrix(y_test.argmax(axis=1), predictions.argmax(axis=1))
precision = np.diag(con) / (np.diag(con) +(con.sum(axis=0) - np.diag(con)))
precision

array([0.69137168, 0.63551402, 0.34088702, 0.6137887 , 0.75759135,
       0.99513163, 0.9383675 , 0.48097826, 0.64133219, 0.86979416])

In [39]:
recall = np.diag(con)/ ((con.sum(axis=1) - np.diag(con)) + np.diag(con))
recall

array([0.34877232, 0.23475259, 0.61078886, 0.61822858, 0.59959267,
       0.98080682, 0.96238736, 0.44305382, 0.45570388, 1.        ])

In [40]:
f1_score = 2 * ((precision * recall) / (precision + recall))
f1_score

array([0.46364985, 0.34285714, 0.43756493, 0.61600064, 0.66939518,
       0.9879173 , 0.95022566, 0.46123779, 0.53281305, 0.93036354])

In [71]:
#PrincipalComponentAnalysis
features = ['id','dur','dwin','proto','djit','swin','smean','state','service','ct_src_dport_ltm','ct_dst_ltm','ct_dst_sport_ltm','ct_src_ltm','ct_flw_http_mthd','ct_srv_dst','dpkts','dloss','dbytes','sttl','dmean','spkts','sbytes','sloss','sinpkt','label']
X2 = df[['id','dur','dwin','proto','djit','swin','smean','state','service','ct_src_dport_ltm','ct_dst_ltm','ct_dst_sport_ltm','ct_src_ltm','ct_flw_http_mthd','ct_srv_dst','dpkts','dloss','dbytes','sttl','dmean','spkts','sbytes','sloss','sinpkt','label']]
Y2 = df[['attack_cat']]
X3 = X2.iloc[:,0:25]  #independent columns
Y3 = Y2.iloc[:,-1]    #target column 

In [72]:
from sklearn.preprocessing import StandardScaler
sScaler = StandardScaler()
rescaleX3 = sScaler.fit_transform(X3)
df_rescaled = pd.DataFrame(data=rescaleX3, columns=features)

In [73]:
X_train, X_test, y_train, y_test = train_test_split(rescaleX3, Y3, test_size=0.3)

In [74]:
y_train = to_categorical(y_train)

In [76]:
#Initialising the Artificial Neural Network
model = Sequential()
#Creating the Input-layer and the first hidden layer
model.add(Dense(250, input_dim=25, activation='sigmoid'))
model.add(Dense(500, input_dim=25, activation='sigmoid'))
model.add(Dense(500, input_dim=25, activation='sigmoid'))
model.add(Dense(250, input_dim=25, activation='sigmoid'))
#Creating the output  layer
model.add(Dense(10, activation='softmax'))
#Compiling the ANN classifier
model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])
model.fit(X_train, y_train, batch_size=50, epochs=100)

Epoch 1/100
81200/81200 [==============================] - 10s 119us/step - loss: 1.1038 - accuracy: 0.5826
Epoch 2/100
81200/81200 [==============================] - 9s 105us/step - loss: 0.8727 - accuracy: 0.6519
Epoch 3/100
81200/81200 [==============================] - 10s 117us/step - loss: 0.8127 - accuracy: 0.6789
Epoch 4/100
81200/81200 [==============================] - 12s 150us/step - loss: 0.7765 - accuracy: 0.6926
Epoch 5/100
81200/81200 [==============================] - 10s 120us/step - loss: 0.7416 - accuracy: 0.7069
Epoch 6/100
81200/81200 [==============================] - 9s 117us/step - loss: 0.7085 - accuracy: 0.7161
Epoch 7/100
81200/81200 [==============================] - 10s 125us/step - loss: 0.6870 - accuracy: 0.7243
Epoch 8/100
81200/81200 [==============================] - 10s 126us/step - loss: 0.6745 - accuracy: 0.7287
Epoch 9/100
81200/81200 [==============================] - 10s 127us/step - loss: 0.6643 - accuracy: 0.7346
Epoch 10/100
81200/81200 [====

In [79]:
predictions = model.predict(X_test)
predictions

array([[1.76865584e-07, 1.41635176e-03, 1.49118630e-02, ...,
        6.61594450e-01, 6.51450595e-12, 5.93226605e-06],
       [3.30842818e-28, 4.10127745e-27, 4.14147878e-19, ...,
        8.56741773e-19, 8.96312044e-24, 2.12786305e-25],
       [2.28544596e-30, 8.62110416e-30, 4.82734592e-19, ...,
        3.23158732e-23, 7.76865960e-24, 1.59759648e-26],
       ...,
       [1.05090339e-05, 4.83202923e-04, 1.71450188e-03, ...,
        9.80655193e-01, 1.32494085e-02, 2.74616219e-09],
       [1.41561434e-01, 2.56661445e-01, 2.85265744e-01, ...,
        3.97389866e-02, 7.55030953e-04, 8.34661265e-08],
       [4.37267119e-07, 1.45802915e-05, 2.07709789e-04, ...,
        1.08573636e-06, 1.15381233e-08, 9.98149276e-01]], dtype=float32)

In [81]:
y_test = to_categorical(y_test)
model.evaluate(X_test,y_test)

34800/34800 [==============================] - 1s 39us/step


[0.5907803846638778, 0.7761494517326355]

In [92]:
pd.crosstab(y_test.argmax(axis=1), predictions.argmax(axis=1), rownames=['True'], colnames=['Predicted'], margins=True)

Predicted,0,1,2,3,4,5,6,7,8,9,All
True,,,,,,,,,,,
0,516,154,857,263,6,0,0,0,0,0,1796
1,8,611,855,210,39,0,0,15,23,3,1764
2,40,156,2065,1014,64,0,0,15,39,7,3400
3,94,196,1597,4009,150,7,0,85,62,48,6248
4,8,14,129,129,1985,0,2,44,79,9,2399
5,1,2,45,31,11,5431,0,1,1,2,5525
6,0,0,0,0,0,0,7721,0,0,0,7721
7,7,32,280,263,155,0,0,1483,153,14,2387
8,0,0,6,106,163,0,0,83,1321,13,1692


In [91]:
metrics.confusion_matrix(y_test.argmax(axis=1), predictions.argmax(axis=1))

array([[ 516,  154,  857,  263,    6,    0,    0,    0,    0,    0],
       [   8,  611,  855,  210,   39,    0,    0,   15,   23,    3],
       [  40,  156, 2065, 1014,   64,    0,    0,   15,   39,    7],
       [  94,  196, 1597, 4009,  150,    7,    0,   85,   62,   48],
       [   8,   14,  129,  129, 1985,    0,    2,   44,   79,    9],
       [   1,    2,   45,   31,   11, 5431,    0,    1,    1,    2],
       [   0,    0,    0,    0,    0,    0, 7721,    0,    0,    0],
       [   7,   32,  280,  263,  155,    0,    0, 1483,  153,   14],
       [   0,    0,    6,  106,  163,    0,    0,   83, 1321,   13],
       [   0,    0,    0,    0,    0,    0,    0,    0,    0, 1868]],
      dtype=int64)

In [87]:
precision = np.diag(con) / (np.diag(con) +(con.sum(axis=0) - np.diag(con)))
precision

array([0.76557864, 0.52446352, 0.35395955, 0.66539419, 0.77147299,
       0.99871276, 0.99974103, 0.85921205, 0.78724672, 0.95112016])

In [88]:
recall = np.diag(con)/ ((con.sum(axis=1) - np.diag(con)) + np.diag(con))
recall

array([0.28730512, 0.34637188, 0.60735294, 0.64164533, 0.8274281 ,
       0.98298643, 1.        , 0.62128194, 0.78073286, 1.        ])

In [89]:
f1_score = 2 * ((precision * recall) / (precision + recall))
f1_score

array([0.41781377, 0.41720724, 0.44726013, 0.653304  , 0.79847144,
       0.99078719, 0.9998705 , 0.72112813, 0.78397626, 0.97494781])